In [ ]:
# %pip install -U torch diffusers transformers accelerate bitsandbytes sentencepiece qwen-vl-utils

#####
# For torchvision: https://pytorch.org/get-started/locally/

# #windows
# %pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126

# #mac
# %pip install torch torchvision

# #linux
# %pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu

In [12]:
import sys
from pathlib import Path
import torch
import json
import gc
from PIL import Image
from pathlib import Path
from tqdm.notebook import tqdm
from huggingface_hub import login
from transformers import (
    AutoProcessor, 
    Qwen2_5_VLForConditionalGeneration, 
    MllamaForConditionalGeneration, # Often used for Llama Vision models
    AutoModelForCausalLM,
    AutoModelForVision2Seq,
    BitsAndBytesConfig
)

In [ ]:
# Add parent directory to path
parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

print("bringing in local modules")
# brining in more modeules because of image generation process
# modules are found in the DataCollection/src folder
# from src import config, gemini_client, data_loader, prompt_builder, output_handler
from src import config, data_loader, prompt_builder, output_handler

print("All modules imported successfully")
print(f"Working directory: {Path.cwd()}")

# Load configuration
# DataCollection/src/config.py ... def load_confi()
# using "generation_config.yaml" for setup
cfg = config.load_config()
# 4-Bit Config (CRITICAL for running 17B Model on 16GB GPU)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

cfg = config.load_config()
data_dir = cfg.get_data_path('generated')


# Login to HuggingFace (Required for Llama 4 Scout)
login(token = 'your token')
# login(token = '')

2026-01-22 21:42:21,756 - src.config - INFO - Loaded environment variables from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\config\.env
2026-01-22 21:42:21,781 - src.config - INFO - Loaded configuration from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\config\generation_config.yaml
2026-01-22 21:42:21,782 - src.config - INFO - Logging configured successfully
2026-01-22 21:42:21,804 - src.config - INFO - Loaded environment variables from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\config\.env
2026-01-22 21:42:21,823 - src.config - INFO - Loaded configuration from c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\config\generation_config.yaml
2026-01-22 21:42:21,829 - src.config - INFO - Logging configur

bringing in local modules
All modules imported successfully
Working directory: c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\notebooks


In [14]:
def analyze_images(handler):
    # Get the directory where images were just saved
    # (handler.images_dir is a Path object from your output_handler.py)
    image_files = sorted(list(handler.images_dir.glob(f"*.{handler.image_format}")))

    if not image_files:
        # Show the last 3 images
        # recent_images = image_files[-3:]
        print("No images found to analyze!")
    return image_files

def save_result(image_name, model_name, result_text, handler):
    """Helper to save analysis to JSON"""
    file_path = handler.analysis_dir / f"{image_name}_{model_name}.json"
    # file_path = data_dir / f"{image_name}_{model_name}.json"
    with open(file_path, 'w') as f:
        json.dump({
            "image": image_name,
            "model": model_name,
            "output": result_text
        }, f, indent=2)

def cleanup_gpu():
    """Force garbage collection to free VRAM for the next model"""
    gc.collect()
    torch.cuda.empty_cache()

In [15]:
def run_qwen(image_files, prompt, handler):
    # MODEL A: QWEN 2.5 VL (OCR Expert)
    print("\nLoading Qwen 2.5 VL")
    model_id = "Qwen/Qwen2.5-VL-7B-Instruct"
    #Fetch model
    processor = AutoProcessor.from_pretrained(model_id)
    model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
        model_id, quantization_config=bnb_config, device_map="auto"
    )

    #iterate through files and run each image through qwen using prompt   
    for img in tqdm(image_files, desc="Qwen Analysis"):
        # Check if done
        if (handler.analysis_dir / f"{img.stem}_qwen_vl.json").exists(): continue
        
        inputs = processor(
            text=[{"type": "image", "image": str(img)}, {"type": "text", "text": prompt}],
            padding=True, return_tensors="pt"
        ).to("cuda")
        
        out = model.generate(**inputs, max_new_tokens=200)
        text = processor.batch_decode(out, skip_special_tokens=True)[0]
        save_result(img, "qwen_vl", text, handler)
        
    del model, processor
    cleanup_gpu()
    return 

In [16]:
def run_phi4(image_files, prompt, handler):
    #good for tagging
    print("\nLoading Phi-4 Multimodal")
    model_id = "microsoft/phi-4-multimodal-instruct"
    processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id, device_map="cuda", torch_dtype="auto", trust_remote_code=True
    )
        
    for img in tqdm(image_files, desc="Phi Analysis"):
        if (handler.analysis_dir / f"{img.stem}_phi4.json").exists(): continue
        
        image = Image.open(img)
        inputs = processor(text=prompt, images=image, return_tensors="pt").to("cuda")
        out = model.generate(**inputs, max_new_tokens=50)
        text = processor.batch_decode(out, skip_special_tokens=True)[0]
        save_result(img, "phi4", text, handler)

    del model, processor
    cleanup_gpu()


In [17]:
def run_llama4_scout(image_files, prompt, handler):
# LLAMA 4 SCOUT (Reasoning)
    print("\n Loading Llama 4 Scout")
    # Note: make sure you have access to this model on Huggingface
    model_id = "meta-llama/Llama-4-Scout-17B-16E" 
    processor = AutoProcessor.from_pretrained(model_id)
    model = AutoModelForVision2Seq.from_pretrained(
        model_id, quantization_config=bnb_config, device_map="auto", trust_remote_code=True
    )
            
    for img in tqdm(image_files, desc="Llama Analysis"):
        if (handler.analysis_dir / f"{img.stem}_llama_scout.json").exists(): continue
        
        # Llama Vision input format
        image = Image.open(img)
        inputs = processor(image, prompt, return_tensors="pt").to("cuda")
        out = model.generate(**inputs, max_new_tokens=150)
        text = processor.decode(out[0], skip_special_tokens=True)
        save_result(img, "llama_scout", text, handler)

    del model, processor
    cleanup_gpu()


In [10]:
#include structured output prompt from larger analysis 
prompt = "Only reply with a json file structure. Your response should have the following json structure:" \
"labels:{ Label_1: please choose the most probable label from this list of labels for the image: 'protest', 'digital image with text in Spanish', 'digital image with text in English', 'a small group of people', 'an illustration or cartoon', 'solidarity', 'an image of a woman and text', 'an image of a man and text', 'a person or selfie', 'a sign(s) or banner(s)', 'statues, landmarks, buildings', 'informational', 'personal belongings or objects', 'image created by bot'" \
"         Label_2: please choose the second most probable label from this list of labels for the image: 'protest', 'digital image with text in Spanish', 'digital image with text in English', 'a small group of people', 'an illustration or cartoon', 'solidarity', 'an image of a woman and text', 'an image of a man and text', 'a person or selfie', 'a sign(s) or banner(s)', 'statues, landmarks, buildings', 'informational', 'personal belongings or objects', 'image created by bot'}" \
"Description: please create a description of the image" \
"Please include blank labels or descriptions if you are unable to provide them. Do not stray from the json structure."
"Please DO NOT include any leading or follow up text or comments, only provide the json file." \


# Based on the output handler module,
# This automatically creates folders in DataCollection/data/generated: images/, metadata/, logs/
handler = output_handler.OutputHandler(
    output_dir=cfg.get_output_path(),  # Uses path from generation_config.yaml
    image_format=cfg.output.get('format', 'png'),
    export_csv=True,
    date_organized=True
)


# Run Analysis
image_files = analyze_images(handler)

#run the mllms to compare
run_qwen(image_files, prompt, handler)
run_phi4(image_files, prompt, handler)
run_llama4_scout(image_files, prompt, handler)


2026-01-22 21:39:03,548 - src.output_handler - INFO - Output directories created at c:\Users\lwert\OneDrive - University of Arizona\Documents\Fellowships\Jetstream\AI-ML_PipelineWorkshop\DataCollection\data\generated


No images found to analyze!

Loading Qwen 2.5 VL


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


ImportError: 
AutoVideoProcessor requires the Torchvision library but it was not found in your environment. Check out the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
